# DETEKSI DAN PENGENALAN WAJAH MENGGUNAKAN KAMERA

### Langkah-Langkah:  1 - Buat Training Data, 2 - Training Model, 3 - Jalankan Pengenalan Wajah


In [1]:
# import library
import numpy as np
import cv2
import os

# 1. Membuat Training Data

In [2]:
# Baca LBP Cascade
lbpcascade_frontalface = 'lbpcascades/lbpcascade_frontalface.xml'

In [3]:
# fungsi deteksi wajah pada opencv
def detek_wajah(input_gbr):
    # konversi gambar ke kabuan
    gbr = cv2.cvtColor(input_gbr, cv2.COLOR_BGR2GRAY)
    
    # klasifikasi cascade
    cascade_wajah = cv2.CascadeClassifier(lbpcascade_frontalface)
    
    # deteksi dengan multiscale 
    wajah = cascade_wajah.detectMultiScale(gbr, scaleFactor=1.2, minNeighbors=3);
    
    #jika tidak ada wajah terdeteksi kembali ke gambar yang asli
    if (len(wajah) == 0):
        return None, None
    
    # ekstrak area wajah
    (x, y, w, h) = wajah[0]
    # kembalikan hanya bagian wajah dari gambar
    return gbr[y:y+w, x:x+h], wajah[0]

In [4]:
# buat kotak persegi untuk deteksi wajah
def gbr_persegi(gbr, persegi):
    # gambar persegi pada wajah yang terdeteksi
    (x, y, w, h) = persegi
    cv2.rectangle(gbr, (x, y), (x+w, y+h), (0, 255, 255), 3)

def gbr_text(gbr, text, x, y):
    # tampilkana tek pada wajah yang terdeteksi
        cv2.putText(gbr, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [23]:
# merekam wajah pada training data

# buka webcam dan baca setiap frame.
video = cv2.VideoCapture(0)
jumlah = 0
while True:
    # baca setiap frame
    ret, frame = video.read()
    # deteksi wajah
    #Setiap frame melewati fungsi detek_wajah
    wajah_pilih, persegi = detek_wajah(frame)
    # hindari wajah terdeteksi
    # jika wawjah terdeteksi,  
    if wajah_pilih is not None:
        jumlah +=1
        # Buat frame
        gbr_persegi(frame, persegi)
        # wajah terdeteksi disimpan dalam folder yang akan digunakan untuk pelatihan.
        cv2.imwrite("rekam_wajah/0/"+str(jumlah)+ ".jpg", wajah_pilih)
    # tampilkan frame     
    cv2.imshow("frame", frame)
    # Escape untuk keluar
    if cv2.waitKey(1)==27:
        break
# release video        
video.release()
cv2.destroyAllWindows() 

# 2. Training Model

In [5]:
# Baca gambar dari folder dan siapkan set pelatihan dan label masing-masing
deteksi_wajah = []
label_wajah = []

def prepare_training_data(path_training_data):
    
    # baca gambar dari folder dan siapkan set data pelatihan
    #  mengembalikan daftar wajah dan label yang terdeteksi
    
    gambar_training = os.listdir(path_training_data)
    
    # tenelusuri setiap direktori dan membaca gambar di dalamnya
    for nama_di_direktori in gambar_training:
        label = int(nama_di_direktori)
        path_training_gbr = path_training_data + "/" + nama_di_direktori
        nama_training_gbr = os.listdir(path_training_gbr)
        
        # menelusuri setiap nama gambar, membaca gambar,
        # deteksi wajah dan tambahkan wajah ke daftar wajah
        for image_name in nama_training_gbr:
            gbr_path = path_training_gbr  + "/" + image_name
            gambar = cv2.imread(gbr_path, 0)
            deteksi_wajah.append(gambar)
            label_wajah.append(label)

    return deteksi_wajah, label_wajah

# siapkan data pelatihan
# data akan berada dalam dua daftar dengan ukuran yang sama
# satu daftar akan berisi semua wajah
# dan daftar lainnya akan berisi label masing-masing untuk setiap wajah
deteksi_wajah, label_wajah = prepare_training_data('rekam_wajah')

# buat pengenal wajah LBPH
lbphfaces_recognizer = cv2.face. LBPHFaceRecognizer_create(radius=2, neighbors=3)

# latih pengenalan wajah dari wajah latihan 
lbphfaces_recognizer.train(deteksi_wajah, np.array(label_wajah))

# print total label dan wajah yang di training
print("Total wajah: ", len(deteksi_wajah))
print("Total label: ", len(label_wajah))
print("Pelatihan Model Berhasil")

Total wajah:  864
Total label:  864
Pelatihan Model Berhasil


# 3. Jalankan Pengenalan Wajah

In [7]:
# buka webcam, baca setiap frame dan berikan ke model terlatih. 
# Untuk prediksi biner: Jika nilai keyakinan prediksi kurang dari nilai ambang batas 
# maka atur teks label sama dengan nama pengguna  selain itu tidak diketahui. 
# Untuk prediksi multikelas: atur label_text ke label
# buka webcam
import datetime as dt
sekarang=dt.datetime.now()
video = cv2.VideoCapture(0)

while True:
    #jalankan video
    ret, frame = video.read()
    
    nama = ['Saya', 'Nanda']
    display_time=sekarang.strftime('%d/%m/%y %H:%M:%S')
    cv2.putText(frame,display_time,(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1)
    # wajah input sama dengan wajah di data-training
    wajah_pilih, persegi = detek_wajah(frame)
    
    if wajah_pilih is not None:
        # nama wajah yang sesuai katagori
        label= lbphfaces_recognizer.predict(wajah_pilih)
        
        # nilai keyakinan prodiksi kurang dari nilai ambang batas maka atur teks label sama dengan nama pengguna
        if label[1]<75:
            # nama teks yang sesuai dengan label
            label_text = nama[label[0]]
        
        # selain itu tidak dikenal
        else: 
            label_text = 'Tidak dikenal'
        # Buat persegi dan teks sebagai penanda    
        gbr_persegi(frame, persegi)
        if label[1]<75:
            gbr_text(frame, label_text, persegi[0], persegi[1]-1)
        else:
            gbr_text(frame, label_text, persegi[0], persegi[1]-5)
    cv2.imshow("frame", frame)
    #escape untuk keluar
    if cv2.waitKey(1)==27:
        break
video.release()
cv2.destroyAllWindows()